Python 3.12.7 ('.venv': venv)

# To use requirements.txt

python -m venv .venv
source .venv/bin/activate  # Mac/Linux
# .venv\Scripts\activate  #  Windows
pip install -r requirements.txt

In [1]:
import sys
import os
import yaml
from ultralytics import YOLO
import torch
import time
import datetime

In [ ]:
# Dataset configuration variables (using an absolute path)
# DATA_CONFIG = "/Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/data_yolov11/data.yaml"
DATA_CONFIG = "/Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/small_yolov12/data.yaml"

print(f"DATA_CONFIG: {DATA_CONFIG}")

DATA_CONFIG: /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/small_yolov12/data.yaml


In [3]:
# Automatically detect the platform
if sys.platform == "darwin":
    PLATFORM = "mac"
elif sys.platform.startswith("win"):
    PLATFORM = "windows"
else:
    PLATFORM = "other"

In [4]:
# Configure device
# Select inference device ("cpu", "cuda" or "mps"), if available:
if PLATFORM == "mac":
    DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
elif PLATFORM == "windows":
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
else:
    DEVICE = "cpu"

# multi-GPU in Windows
MULTI_GPU = False  # Change to True if you want multi-GPU training in Windows
if MULTI_GPU and PLATFORM == "windows":
    TRAIN_DEVICE = [0, 1]
else:
    TRAIN_DEVICE = DEVICE

print(f"Executing in {PLATFORM}. Using device: {TRAIN_DEVICE}")

# Clean cache if using CUDA (Windows)
if DEVICE == "cuda":
    torch.cuda.empty_cache()

Executing in mac. Using device: mps


In [5]:
print("torch version:", torch.__version__)

if DEVICE == "cuda":
    print("CUDA version:", torch.version.cuda)
    print("CUDA available:", torch.cuda.is_available())
elif DEVICE == "mps":
    print("MPS available:", torch.backends.mps.is_available())
    print("MPS built:", torch.backends.mps.is_built())
else:
    print("Executing in CPU")

torch version: 2.6.0
MPS available: True
MPS built: True


In [ ]:
# You can download any version of yolo, you can go to the yolo github
# model = YOLO("yolo11x.pt")

model = YOLO("yolov8n.pt")

In [13]:
# Check if the dataset configuration file exists
if not os.path.exists(DATA_CONFIG):
    raise FileNotFoundError(f"Dataset configuration file '{DATA_CONFIG}' does not exist. "
                            "Please provide a valid path.")

# Start timer
start_time = time.time()

# Train model
results = model.train(
    data=DATA_CONFIG,
    epochs=100,             # Adjust the number of epochs; using 5 for testing
    rect=True,              # Rectangular training
    imgsz=960,              # Image size
    device=TRAIN_DEVICE,    # Use the configured device (will be "mps" on Mac)
    batch=16,               # Batch size; adjust based on your GPU/CPU capacity
    workers=10,             # Number of workers; adjust based on your CPU capacity
    augment=True,           # Enable data augmentations (ensure your YAML file is properly configured)
    amp=True,
    #multi_scale=True,      # Uncomment to try training with variable image sizes
    #optimizer="AdamW",     # Optionally change the optimizer if needed
    conf=0.025,
    lr0=0.0001, lrf=0.0005, # Adjust the learning rate
    iou=0.5,                # IoU threshold
    patience=30             # Early stopping
    #mosaic=1.0, mixup=0.7          
)

# End timer and compute elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

# Format elapsed time as hh:mm:ss
formatted_time = str(datetime.timedelta(seconds=round(elapsed_time)))
print(f"Training completed in {formatted_time}.")

Ultralytics 8.3.95 🚀 Python-3.12.7 torch-2.6.0 MPS (Apple M2 Max)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/small_yolov12/data.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=mps, workers=10, project=None, name=train162222, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.025, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/small_yolov12/train/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]

WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/small_yolov12/valid/labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]

Plotting labels to /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/runs/detect/train162222/labels.jpg... 


...

    100/100      69.7G        nan        nan        nan         56        960: 100%|██████████| 4/4 [00:23<00:00,  5.75s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.10it/s]

                   all          3         27          0          0          0          0



100 epochs completed in 0.658 hours.
Optimizer stripped from /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/runs/detect/train162222/weights/last.pt, 114.4MB
Optimizer stripped from /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/runs/detect/train162222/weights/best.pt, 114.4MB

Validating /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/runs/detect/train162222/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.12.7 torch-2.6.0 MPS (Apple M2 Max)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


                   all          3         27          0          0          0          0
Speed: 14.9ms preprocess, 145.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI 599A/Code/CSCI_E-599a/runs/detect/train162222
Training completed in 0:39:36.


In [8]:
import os
print(os.cpu_count())

12
